# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [64]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [24]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [25]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [26]:
symbol = 'MSFT'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Microsoft Corporation',
 'marketcap': 1795670304810,
 'week52high': 249.55,
 'week52low': 138.22,
 'week52change': 0.39479479996682754,
 'sharesOutstanding': 7705718490,
 'float': 0,
 'avg10Volume': 46461506,
 'avg30Volume': 35580096,
 'day200MovingAvg': 220.28,
 'day50MovingAvg': 228.09,
 'employees': 0,
 'ttmEPS': 6.95,
 'ttmDividendRate': 2.1385453134991694,
 'dividendYield': 0.00927913559313995,
 'nextDividendDate': '2021-02-05',
 'exDividendDate': '2020-11-13',
 'nextEarningsDate': '2021-01-16',
 'peRatio': 34.2942709779296,
 'beta': 1.184007397013103,
 'maxChangePercent': 9.204566908189332,
 'year5ChangePercent': 3.6328834889535204,
 'year2ChangePercent': 1.2949924602599783,
 'year1ChangePercent': 0.3671726181111398,
 'ytdChangePercent': 0.04396279510672485,
 'month6ChangePercent': 0.1478062806622659,
 'month3ChangePercent': 0.1495869436913713,
 'month1ChangePercent': 0.04845637556331168,
 'day30ChangePercent': 0.04435966256888314,
 'day5ChangePercent': 0.0278689

In [27]:
data['year1ChangePercent']

0.3671726181111398

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [28]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', '1-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [29]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Price,1-Year Price Return,Number of Shares to Buy
0,A,124.21,0.436285,N/A
1,AAL,17.36,-0.391021,N/A
2,AAP,153.55,0.111823,N/A
3,AAPL,134.67,0.661325,N/A
4,ABBV,104.40,0.334368,N/A
...,...,...,...,...
95,CINF,87.68,-0.190184,N/A
96,CL,79.00,0.154498,N/A
97,CLX,216.68,0.388828,N/A
98,CMA,58.80,-0.022579,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [30]:
final_dataframe.sort_values('1-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe.head(10)
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,1-Year Price Return,Number of Shares to Buy
0,78,CARR,40.00,2.240759,N/A
1,23,ALB,163.21,1.075212,N/A
2,24,ALGN,541.95,1.047892,N/A
3,6,ABMD,357.40,0.875384,N/A
4,85,CDNS,135.54,0.805436,N/A
5,31,AMD,87.63,0.769708,N/A
6,36,AMZN,3275.30,0.722461,N/A
7,3,AAPL,134.67,0.661325,N/A
8,29,AMAT,99.78,0.630717,N/A
9,63,BIO,597.97,0.569168,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [32]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        portfolio_size = abs(float(portfolio_size))
    except ValueError:
        print("That's not a number! \nPlease try again")
        portfolio_size = input('Enter the value of your portfolio:')
        portfolio_size = abs(float(portfolio_size)) # Value automatically converted to a positive
        
portfolio_input()
print(portfolio_size)

Enter the value of your portfolio: 10000000


10000000.0


In [33]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
    
final_dataframe

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,1-Year Price Return,Number of Shares to Buy
0,78,CARR,40.00,2.240759,25000
1,23,ALB,163.21,1.075212,6127
2,24,ALGN,541.95,1.047892,1845
3,6,ABMD,357.40,0.875384,2797
4,85,CDNS,135.54,0.805436,7377
5,31,AMD,87.63,0.769708,11411
6,36,AMZN,3275.30,0.722461,305
7,3,AAPL,134.67,0.661325,7425
8,29,AMAT,99.78,0.630717,10022
9,63,BIO,597.97,0.569168,1672


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [56]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    '1yr Price Return',
    '1yr Return Percentile',
    '6mo Price Return',
    '6mo Return Percentile',
    '3mo Price Return',
    '3mo Return Percentile',
    '1mo Price Return',
    '1mo Return Percentile'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'n/a',
                    data[symbol]['stats']['year1ChangePercent'],
                    'n/a',
                    data[symbol]['stats']['month6ChangePercent'],
                    'n/a',
                    data[symbol]['stats']['month3ChangePercent'],
                    'n/a',
                    data[symbol]['stats']['month1ChangePercent'],
                    'n/a'
                ],
                index = hqm_columns
            ),
            ignore_index = True
        )

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,1yr Price Return,1yr Return Percentile,6mo Price Return,6mo Return Percentile,3mo Price Return,3mo Return Percentile,1mo Price Return,1mo Return Percentile
0,A,123.61,n/a,0.456509,n/a,0.264935,n/a,0.185998,n/a,0.0259739,n/a
1,AAL,17.81,n/a,-0.389605,n/a,0.556546,n/a,0.53341,n/a,0.065875,n/a
2,AAP,153.70,n/a,0.116121,n/a,-0.021313,n/a,0.0146538,n/a,-0.055681,n/a
3,AAPL,136.60,n/a,0.666113,n/a,0.381118,n/a,0.21515,n/a,-0.0133452,n/a
4,ABBV,103.10,n/a,0.32837,n/a,0.0975649,n/a,0.218997,n/a,-0.0152885,n/a
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.61,n/a,-0.0303699,n/a,0.120009,n/a,0.0954696,n/a,-0.0758772,n/a
501,ZBH,158.89,n/a,0.0255501,n/a,0.116812,n/a,0.168261,n/a,0.00967421,n/a
502,ZBRA,398.28,n/a,0.599587,n/a,0.416801,n/a,0.371062,n/a,0.0109176,n/a
503,ZION,45.70,n/a,-0.0213521,n/a,0.388194,n/a,0.394016,n/a,0.0214102,n/a


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [61]:
# There seems to be some 'None' values in the dataframe that could cause problems.

hqm_dataframe[hqm_dataframe.isna().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,1yr Price Return,1yr Return Percentile,6mo Price Return,6mo Return Percentile,3mo Price Return,3mo Return Percentile,1mo Price Return,1mo Return Percentile


In [66]:
time_periods = [
                '1yr',
                '6mo',
                '3mo',
                '1mo'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        # Replace 'None' values with 0
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
        
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],
                                                       hqm_dataframe.loc[row, change_col])
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,1yr Price Return,1yr Return Percentile,6mo Price Return,6mo Return Percentile,3mo Price Return,3mo Return Percentile,1mo Price Return,1mo Return Percentile
0,A,123.61,n/a,0.456509,87.9208,0.264935,67.5248,0.185998,59.4059,0.0259739,71.2871
1,AAL,17.81,n/a,-0.389605,2.77228,0.556546,93.2673,0.53341,95.2475,0.065875,83.7624
2,AAP,153.70,n/a,0.116121,55.6436,-0.021313,13.6634,0.0146538,15.4455,-0.055681,22.5743
3,AAPL,136.60,n/a,0.666113,95.8416,0.381118,82.1782,0.21515,64.9505,-0.0133452,44.9505
4,ABBV,103.10,n/a,0.32837,82.1782,0.0975649,36.2376,0.218997,65.3465,-0.0152885,43.9604
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.61,n/a,-0.0303699,34.8515,0.120009,41.7822,0.0954696,35.0495,-0.0758772,12.2772
501,ZBH,158.89,n/a,0.0255501,43.5644,0.116812,40.396,0.168261,56.2376,0.00967421,62.1782
502,ZBRA,398.28,n/a,0.599587,94.0594,0.416801,84.5545,0.371062,83.7624,0.0109176,63.1683
503,ZION,45.70,n/a,-0.0213521,36.4356,0.388194,82.9703,0.394016,86.3366,0.0214102,68.5149


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: